# Computer assignment 2 - AI
# Genetics Algorithm
==========================================================================================================================

## Introduction
In this project we use mthods based on nature and natural selection. <br>
The methods are called Geneteics algorithm. The algorithm is inspired <br>
by Natural concepts like mating, genetic mutation and natural selection.<br>
In Genetic algorithm there are points that need to be considered. <br>
Insufficient population may cause poor operation of algorithm. Actually<br>
to reach the complete answers population size should be coherent to <br>
number of possible answers. The smaller the population the greater the<br>
likehood of omitting of variety. But increasing population without <br>
attention may cause poor performance of algorithm. <br>
So there should be a equilibrium point for this.<br>
From a practical point of view, Genetics algorithms are used in problems<br>
with huge space of states. <br>
## Project description
In this project we to find gates of a logic circuit using Genetics algorithms<br>
In such a way that its output corresponds to the truth table. The circuit is<br>
made using 9 gates and we are to find those 9 gates. <br>
The truth table is a csv file which has 11 columns and 1024 rows. First 10<br>
columns are inputs of circuit and last column is its output. <br>
We have 1024 rows because by 10 inputs 2^10 different cases are possible<br>
for circuit inputs.
## Phase 1
### Genetics concepts in this project
### Chromosome
The answer of this problem is list consecutive gates<br>
A chromosome describe a possible answer for our problem. <br>
So a chromosome made by 9 consiquence gates. Number of <br>
chromosomes construct the population.<br>
### Gene
Every chromosome is made by number of genes. <br>
A gene represent a property<br> 
In our case every gene in a chromosome represent<br>
a gate in the ciruit. Actually every gate is considered<br>
as a property of the answer.

## Phase 2
### Creating initial population
Generating initial population is the first action in genetic<br>
algorithm. Each chromosome in the population is made randomly.<br>
And no bias existed. The population contain 1000 chromosomes.<br>
This number is a equilibrium point. Its huge enough to be sure<br>
that algorithm find the answer and does not stuck in local maximums.<br>

## Phase 3
### Fitness
For every chromosome its fitness is number of rows(in truth table)<br>
which their output is same as output of chromosome gates(genes) <br>
Our desired chromsome has fitness of 1024.<br>

## Phase 4
### Crossover
Crossover is representation of mating in natural case. <br>
In crossover by specified probability parents (2 chromosome)<br>
create 2 children. Each child inherit some genes from parent 1<br>
and some from parent 2. If child 1 inherit a specific gene<br>
from parent 1, child 2 inherit them from parent 2.
### Mutation
Mutation is represent of mutation in natural case. <br>
In mutation every chromosome by a specific change <br>
sees one of its genes altered. <br>

## Phase 5
### Questions
### 1
Fitness of a chromosome describe how much ideal it is <br>
By fitnesses desired chromsomes and unwilling chromosomes<br>
are distinguishable. Number of matched outputs of a chromsome<br>
by truth table describe how good it is.

### 2
We do ranked-based selection in this algorithm. Values of<br>
fitnesses of chromosomes may differ considerably and choosing<br>
it as weight for probability in selection may cause biased <br>
selection and not achieving varity. In fact it may cause some<br>
chromosomes to get a very big probability to be select compared<br>
to the rest.<br>

### 3
Crossover and mutation can evolve the population and emerge <br>
strong and diverse candidates. <br>
#### Probability of chrossover:
average of fitnesses of population divided by population size.<br>
##### reason:
As the population evolves average of fitnesses of it increase <br>
and stronger candidates appear. So probability of crossover <br>
decrease to not losing strong candidates.
#### Probability of mutation:
1 divided by population size.
##### reason:
The Pprobability of mutation is highly affected by population size.<br>
The bigger the population the more number of various candidates. <br>
So the chance of mutation becomes less. And the smaller the population<br>
the less number of various candidates. So we need diverse candidates to <br>
be emerged.

### 4
Sometimes our population evolves and reach a high average fitness<br>
but they are not divers and the algorithm does not find a way to <br>
final optimal answer.
To solve this problem increasing population could be helpfull to bring<br>
diversity to population. Increasing crossover probability and mutation<br>
probability could be helpfull to. It may lead the population to have <br>
various candidates.



In [13]:
import random
import pandas as pd
import copy
import time

def gate(operator, op1, op2):
    if operator == 0:
        return op1 and op2
    elif operator == 1:
        return op1 or op2
    elif operator == 2:
        return not (op1 and op2)
    elif operator == 3:
        return not (op1 or op2)
    elif operator == 4:
        return (op1 and (not op2)) or ((not op1) and op2)
    elif operator == 5:
        return not ((op1 and (not op2)) or ((not op1) and op2))
    else:
        return False
def print_answer(answer):
    print("Answer: ", end=' ')
    for i in range(len(answer)):
        if answer[i] == 0:
            print("AND", end=' ')
        elif answer[i] == 1:
            print("OR", end=' ')
        elif answer[i] == 2:
            print("NAND", end=' ')
        elif answer[i] == 3:
            print("NOR", end=' ')
        elif answer[i] == 4:
            print("XOR", end=' ')
        elif answer[i] == 5:
            print("XNOR", end=' ')
        else:
            return False
    
    
class GeneticAlgorithm:
    def __init__(self):
        self.population = []
        self.population_size = 1000
        self.pool = []
        self.truth_table = pd.read_csv('truth_table.csv')
        self.truth_table.replace('TRUE', True)
        self.truth_table.replace('FALSE', False)
        self.truth_table = self.truth_table.values.tolist()
        self.num_of_rows = len(self.truth_table)
        self.num_of_inputs = len(self.truth_table[0]) - 1
        self.gate_number = self.num_of_inputs - 1

    def fitness(self, chromosome):
        chromosome_fitness = 0

        for i in range(self.num_of_rows):
            output = gate(chromosome[0], self.truth_table[i][0], self.truth_table[i][1])
            for j in range(1, len(chromosome)):
                output = gate(chromosome[j], output, self.truth_table[i][j + 1])
            
            if output == self.truth_table[i][self.num_of_inputs]:
                chromosome_fitness += 1

        return chromosome_fitness
        
    def generate_initial_population(self):
        for i in range(self.population_size):
            new_chromosome = random.choices(range(6), k=self.gate_number)
            new_fitness = self.fitness(new_chromosome)
            self.population.append((new_chromosome, new_fitness))
    
    def selection(self):
        self.pool = []
        self.population.sort(key=lambda x:x[1])

        for i in range(self.population_size):
            self.pool.append(copy.deepcopy(random.choices(self.population, weights=list(range(1,self.population_size + 1)), k=1)[0]))
            
        self.population = []
    
    def crossover(self):
        while self.pool:
            parent1 = self.pool.pop()
            parent2 = self.pool.pop()
            if random.randint(0, self.num_of_rows) > sum(map(lambda x: x[1], self.population)) / self.population_size:
                self.population.append(parent1)
                self.population.append(parent2)
            else:
                index = random.randint(1, self.num_of_inputs - 3)
                child1 = parent1[0][:index] + parent2[0][index:]
                child2 = parent2[0][:index] + parent1[0][index:]
                self.population.append((child1, self.fitness(child1)))
                self.population.append((child2, self.fitness(child2)))
    
    def mutation(self):
        for i in range(self.population_size):
            if random.random() < 1 / self.population_size:
                self.population[i][0][random.randint(0,self.num_of_inputs - 2)] = random.randint(0,5)
    
    def success(self):
        for i in range(self.population_size):
            if self.population[i][1] == self.num_of_rows:
                print("Success!")
                print_answer(self.population[i][0])
                print("\nfitness: ", self.fitness(self.population[i][0]))
                return True
        return False
    
    def evolution_cycle(self):
        self.selection()
        self.crossover()
        self.mutation()
        
    def algorithm(self):
        self.generate_initial_population()

        counter = 0
        while True:
            self.evolution_cycle()
            
            counter += 1
            print("Evolution number: ", counter)
            print("Fitness average: ", sum(map(lambda x: x[1], self.population)) / self.population_size)
            print('<---------------------------->')
            if self.success():
                break
        

GA = GeneticAlgorithm()
time1 = time.time()
GA.algorithm()
time2 = time.time()
print("Time: ", time2 - time1, " s")
print("Finish")


Evolution number:  1
Fitness average:  619.195
<---------------------------->
Evolution number:  2
Fitness average:  700.897
<---------------------------->
Evolution number:  3
Fitness average:  757.63
<---------------------------->
Evolution number:  4
Fitness average:  807.724
<---------------------------->
Evolution number:  5
Fitness average:  848.343
<---------------------------->
Evolution number:  6
Fitness average:  878.739
<---------------------------->
Evolution number:  7
Fitness average:  907.186
<---------------------------->
Evolution number:  8
Fitness average:  926.333
<---------------------------->
Evolution number:  9
Fitness average:  938.386
<---------------------------->
Evolution number:  10
Fitness average:  952.283
<---------------------------->
Evolution number:  11
Fitness average:  958.596
<---------------------------->
Evolution number:  12
Fitness average:  963.118
<---------------------------->
Success!
Answer:  NAND XNOR OR XNOR AND OR AND XOR NOR 
fitnes